In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

cuda


In [34]:
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(4096, 10)
vgg16.to(device)
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [35]:
# optimizer
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [36]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [37]:
# Validation Function
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in tqdm(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    val_loss = val_running_loss / len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_dataloader.dataset)
    
    # Calculate metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1

In [38]:
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for  data, target in tqdm(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [39]:
import torch
import torch.nn.functional as F

def majority_vote_prediction(model, images, transformations, device):
    """
    Given a batch of images and a list of transformations,
    compute the model's predictions on:
        1) the original images,
        2) each transformed version,
    then take a majority vote across all predictions for each sample.
    
    Returns a tensor of shape [batch_size] with the final voted class.
    """
    model.eval()

    # 1) Original predictions
    with torch.no_grad():
        original_out = model(images)
        _, original_preds = torch.max(F.softmax(original_out, dim=1), dim=1)  # shape [batch_size]

    # 2) Collect predictions in a list
    all_preds = [original_preds]

    # 3) For each metamorphic transform
    for tf in transformations:
        # apply transform on CPU, then move back to device
        x_tf = tf(images.cpu()).to(device)

        with torch.no_grad():
            out_tf = model(x_tf)
            _, preds_tf = torch.max(F.softmax(out_tf, dim=1), dim=1)

        all_preds.append(preds_tf)

    # 4) Stack them into shape [num_transforms+1, batch_size]
    stacked_preds = torch.stack(all_preds, dim=0)

    # 5) Majority vote across the first dimension
    voted_preds = stacked_preds.mode(dim=0).values  # shape [batch_size]
    return voted_preds

In [40]:
from tqdm import tqdm

def test_with_majority_vote(model, loader, transformations, device):
    """
    Evaluates the model on 'loader' using majority-voted predictions (original + transforms).
    Prints per-batch accuracy and returns the final overall accuracy.
    """
    model.eval()
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(tqdm(loader)):
        images, labels = images.to(device), labels.to(device)

        # Get majority-voted predictions for this batch
        voted_preds = majority_vote_prediction(model, images, transformations, device)

        # Compare to ground-truth
        batch_correct = (voted_preds == labels).sum().item()
        batch_size = labels.size(0)

        correct += batch_correct
        total += batch_size

        # Print a line for each batch
        print(f"Batch {i} | MajVot Accuracy: {100.0 * batch_correct / batch_size:.2f}% | Batch Size: {batch_size}")

    # Final overall accuracy
    overall_accuracy = 100.0 * correct / total
    print(f"\nFinal MajVot Accuracy (across all test samples): {overall_accuracy:.2f}%")
    return overall_accuracy


In [43]:
majvot_acc = test_with_majority_vote(vgg16, testloader, transformations, device)
print(f"Majority Vote final accuracy: {majvot_acc:.2f}%")

  0%|▎                                                                                     | 1/313 [00:00<02:28,  2.10it/s]

Batch 0 | MajVot Accuracy: 96.88% | Batch Size: 32


  1%|▌                                                                                     | 2/313 [00:00<02:23,  2.17it/s]

Batch 1 | MajVot Accuracy: 75.00% | Batch Size: 32


  1%|▊                                                                                     | 3/313 [00:01<02:18,  2.25it/s]

...

Batch 310 | MajVot Accuracy: 81.25% | Batch Size: 32


100%|███████████████████████████████████████████████████████████████████████████████████▋| 312/313 [02:14<00:00,  2.33it/s]

Batch 311 | MajVot Accuracy: 81.25% | Batch Size: 32


100%|████████████████████████████████████████████████████████████████████████████████████| 313/313 [02:15<00:00,  2.32it/s]

Batch 312 | MajVot Accuracy: 62.50% | Batch Size: 16

Final MajVot Accuracy (across all test samples): 81.90%
Majority Vote final accuracy: 81.90%
